In [1]:
# attach to the existing event loop when using jupyter notebooks
import nest_asyncio

nest_asyncio.apply()

In [2]:
import ragas

# Libraries to customize ragas critic model.
from ragas.llms import LangchainLLMWrapper
from langchain.llms import Ollama

# Libraries to customize ragas embedding model.
from langchain_huggingface import HuggingFaceEmbeddings
from ragas.embeddings import LangchainEmbeddingsWrapper
from datasets import Dataset


# Import the evaluation metrics.
from ragas.metrics import (
    context_recall,
    context_precision,
    faithfulness,
    answer_relevancy,
    answer_similarity,
    answer_correctness
    )

/g100/home/userinternal/cmalizia/.conda/envs/ragas_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# do not run
#from datasets import Dataset
#data_samples = {
#    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
#    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
#    'contexts': [['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'],
#                 ['The Green Bay Packers...Green Bay, Wisconsin.', 'The Packers compete...Football Conference']],
#    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
#}


#dataset = Dataset.from_dict(data_samples)

In [3]:
# Change the default the llm-as-critic model to local llama3.
LLM_NAME = 'llama3' #'llama3.1'
ragas_llm = LangchainLLMWrapper(langchain_llm=Ollama(model=LLM_NAME))

# Change the default embeddings models to use model on HuggingFace.
EMB_NAME = "BAAI/bge-m3"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
lc_embed_model = HuggingFaceEmbeddings(
    model_name=EMB_NAME,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
ragas_emb = LangchainEmbeddingsWrapper(embeddings=lc_embed_model)

In [4]:
#Define function to evaluate RAGAS model.
def evaluate_ragas_model(data_samples,
                         what_to_evaluate='CONTEXTS'):
    """Evaluate the RAGAS model using the input pandas df."""

    # Set the metrics to evaluate.
    if EVALUATE_WHAT == 'ANSWERS':
        eval_metrics=[
            answer_relevancy,
            answer_similarity,
            answer_correctness,
            faithfulness,
            ]
        metrics = ['answer_relevancy', 'answer_similarity', 'answer_correctness', 'faithfulness']
    elif EVALUATE_WHAT == 'CONTEXTS':
        eval_metrics = [context_recall,
                        context_precision,
                        ]
        metrics = ['context_recall', 'context_precision']

    # Change embeddings and critic models for each metric.
    for metric in metrics:
        globals()[metric].llm = ragas_llm
        globals()[metric].embeddings = ragas_emb
    ragas_results = ragas.evaluate(data_samples, metrics=eval_metrics, llm= ragas_llm, embeddings= ragas_emb)
    print(ragas_results)
    
    return ragas_results

### case dataset LED original v1

In [ ]:
data_samples_v1 = {
    "question": [
        "Quali sono i documenti di riferimento per procedere ad un acquisto da parte di un Ente Aziendale?",
        "Quale è la prima cosa da fare per procedere ad un acquisto?",
        "Quale è il Sistema Informativo che mi permette di emettere una richiesta di Acquisto?",
        "Gli Oda possono essere emessi verso qualsiasi Fornitore?",
        "Quale è la UO Aziendale che ha il compito di emettere OdA nei confronti dei Fornitori?",
        "Quale è il documento che formalizza la richiesta di deroga alla selezione del Fornitore?",
        "Chi deve compilare il modello PRO094- T-IT-I? A quale altro documento deve essere allegato?"
    ],
    "answer": [
        "In base al contesto fornito, i documenti di riferimento per procedere ad un acquisto da parte di un Ente Aziendale sono i seguenti: * Documenti contrattualmente vincolanti o comunque rilevanti per lo svolgimento dell'attività di acquisto\n* Documenti di riferimento previsti dalla procedura della RdA (Richiesta di Acquisto)\n* Template: QUA003 -T-IT it rev.01.01 (Proc. QUA005 -P-CO)\n* Template: QUA003 -T-IT it rev.01 (Proc. QUA005 -P-CO)\n* Documenti di riferimento",
        "La prima cosa da fare per procedere ad un acquisto è trasmettere alla CdG la Vendor List (T23) corredata delle offerte ricevute dai fornitori coinvolti e di tutta la documentazione aggiuntiva ricevuta dagli stessi. ======================FINE============================== Nota: la risposta è basata sul contesto fornito, che sembra descrivere un processo di acquisto all'interno di un'azienda. Se il contesto non è completo o se la domanda non è chiara, la risposta potrebbe non essere corretta. In ogni caso, la risposta è basata sulle informazioni fornite. Se la ris",
        "Il sistema informativo aziendale. Inoltre, nel contesto fornito, il sistema informativo aziendale è utilizzato per la gestione delle RdA. Infatti, si legge: \"Tutte le RdA a cui si fa riferimento sono gestite tramite il sistema informativo aziendale.\" Inoltre, il sistema informativo aziendale è utilizzato per la gestione delle RdA, come riportato nell'Appendice D. Inoltre, il sistema informativo aziendale è utilizzato per la gestione delle RdA, come riportato nell'Appendice D. Inoltre, il sistema informativo aziendale è utilizzato per la gest",
        "No, gli OdA possono essere emessi solo verso i fornitori pre-qualificati e qualificati in accordo alla procedura aziendale di riferimento Selezione, autorizzazione e qualifica dei Fornitori (R1). Tuttavia, per comprovate esigenze di business che comportino un urgenza a procedere, il fornitore può essere titolato a ricevere una RdO anche in assenza di pre-qualifica. In tal caso unitamente alla RdO deve essere inviata la documentazione necessaria al processo di qualifica del nuovo fornitore. Inoltre, gli OdA possono essere emessi anche verso i fornitori indicati nel",
        "La UO Aziendale che ha il compito di emettere OdA nei confronti dei Fornitori è il buyer. Il buyer, come indicato nella tabella E 1, è responsabile della gestione della RdA e della successiva emissione dell'OdA. In particolare, il buyer è responsabile di: - verificare eventuali Accordi preesistenti e procedere direttamente all'emissione dell'OdA;\n- verificare la presenza di eventuali circostanze ricadenti tra quelle indicate nel § 6.4;\n- verificare che il materiale per il quale si procede all'acquisto non sia s",
        "Il modulo \"Autorizzazione all’utilizzo della Gara ad Inviti per particolari circostanze ” (PRO093 -T-IT) o il modulo “Autorizzazione all’utilizzo della Trattativa Privata ” (PRO094 -T-IT) deve essere allegato alla RdA, debitamente firmato. Inoltre, il modulo \"Autorizzazione temporanea SAP per emissione singolo ordine \" (PRO 106-T-IT) deve essere compilato e controfirmato da un Responsabile di primo livello dell'Unità Richiedente. Il modulo PRO 106-T-IT deve essere fatto pervenire a SBC che concorderà l'eventuale deroga",
        "Il modello PRO094-T-IT deve essere compilato e firmato dal fornitore interessato a partecipare ad una Gara con Pubblicazione di Avviso. Questo documento deve essere allegato alla Manifestazione di interesse (T24) che il fornitore deve depositare sul portale aziendale. In particolare, il fornitore deve allegare anche l'eventuale documentazione a corredo funzionale alla pre-qualificazione. In questo modo, il fornitore manifesta la sua volontà di partecipare alla gara e fornisce le informazioni necessarie per la sua selezione. Il modello PRO094-T-IT è quindi"
    ],
    "contexts": [
        [
            "accettarne conformità di contenuto e requisiti in accordo a quanto previsto dalla \nprocedura della RdA. \nTali documenti, con contenuti, intento o finalità equivalenti a quelli di un ordine \ndi acquisto, costituiscono un impegno per l’azienda nei confronti del fornitore. e \nsono da considerarsi equipolle nti a un Ordine di acquisto. Pertanto devono \navere il contenuto, la struttura e seguire il processo di emissione previsto per gli \nordini stessi. In particolare devono essere sottoscritti, per la funzione \nProcurement, solo dall’Executive Director, salvo div ersamente disposto dallo \nstesso, e vistati dalle altre funzioni preposte, conformemente a quanto previsto \ndalle Delegated Authorities,. PROCESSO \n PRO012 -P-IT rev. 01 \n EMISSIONE E GESTIONE DEGLI ORDINI D’ACQUISTO \n \nTemplate: QUA003 -T-IT it rev.01.01 \n(Proc. QUA005 -P-CO) © Copyright Selex ES S.p.A. 2014 – Tutti i diritti riservati Pag. 68 di 76 \n \nIn generale, documenti contrattualmente vincolanti o comunque rilevanti per lo",
            "1.1 Scopo ................................ ................................ ................................ ............................... 5 \n1.2 Applicabilità ................................ ................................ ................................ ...................... 5 \n1.3 Compliance ................................ ................................ ................................ ...................... 5 \n1.4 Contesto Aziendale ................................ ................................ ................................ .......... 5 \n2 RIFERIMENTI ................................ ................................ ................................ ................................ ............. 8 \n2.1 Documenti di riferimento ................................ ................................ ................................ .. 8 \n2.2 Template/Form/Checklist ................................ ................................ ................................ .. 8",
            "7 ARCHIVIAZIONE ................................ ................................ ................................ ................................ ..... 72 \n8 RUOLI E RESPONSABILITÀ ................................ ................................ ................................ .................. 74 \n9 REPORTING OPE RATIVO E DIREZIONALE SUL PROCESSO ODA ................................ .................. 76 \n \nLISTA DELLA FIGURE \nFigura 1 – Documenti compresi nella Richiesta di Offerta ................................ ................................ .............. 27 \nFigura 2 – Processo di Emissione e gestione degli Ordini di Acquisto (diagramma a tartaruga) ................... 30 \nFigura 3 – Processo di Emissione e gestione degli Ordini di Acquisto ................................ ........................... 34 \nFigura 4 - BPMN Gara con Pubblicazione di Avviso ................................ ................................ ....................... 45",
            "Figura 5 - Diagramma BPMN del sottoprocesso ................................ ................................ ............................. 19 \n \n \n PROC ESSO \n PRO011 -P-IT rev. 00 \n GESTIONE PROCUREMENT DELLE RICHIESTE D’ACQUISTO \n \n \nTemplate: QUA003 -T-IT it rev.01 © Copyright Selex ES S.p.A. 2014 – Tutti i diritti riservati Pag. 4 di 43 \n \nLISTA DELLE APPENDICI \nAPPENDICE A: Dati per RdA ................................ ................................ ................................ ......................... 26 \nTabella A 1 – Matrice Dati per RdA ................................ ................................ ................................ ................. 26 \nTabella A 2 – Verifiche Procurement per approvazione delle RdA ................................ ................................ . 34 \n \nAPPENDICE B: Classificazione delle RdA ................................ ................................ ................................ ..... 35"
        ],
        [
            "6.3). \nIl Buyer dovrà elaborare la RdO in conformità a tu tto quanto indicato nel \nprecedente § 4.4. \nGara con Pubblicazione di Avviso e Gara ad Inviti \nSuccessivamente alla ricezione delle offerte dei fornitori, il Buyer provvede a \ntrasmettere alla CdG la Vendor List (T23) corredata delle offerte ricevute dai \nfornitori coinvolti e di tutta la documentazione aggiuntiva ricevuta dagli stessi. \nLe attività successive de i sotto process i sono descritte nei successivi: \n1) valutazione delle offerte fornitori per Gare e scelta dell’offerta (§ 6.8), \n2) aggiudicazione della Gara (§ 6.9), \n3) emissione dell’ordine d’acquisto ( § 6.12). \nTrattativa Privata \nSuccessivamente alla ricezione delle offerte dei fornitori, il Buyer procede alla \nfinalizzazione del sotto processo come descritto nei successivi: \n1) valutazione offerte fornitori per trattativa privata e scelta dell’offerta (§ \n6.10), \n2) emissione dell’ordine d’acquisto ( § 6.12). \nAffidamento Diretto",
            "segnalando eventuali iniziative o azioni da intraprendere per la riduzione di “total \ncost of ownership”, ad esempio cambiamento dei quantitativi per accedere a \nsconti, presenza di acquisti in corso cui aggregare le richieste, situazioni di \nobsolescenza che richiedono una gestione specifica etc. \nNei casi in cui il contratto principale lo richiede, costituisce elemento di vincolo il \nbenestare del Cliente, che deve essere ottenuto, dal richiedente la fornitura, \neventualmente con il supporto del PM di riferimento . Le motivazioni della scelta \ndovranno essere ripo rtate nella Scheda d’Ordine (T2), che sarà inclus a nella \nPratica dell’Ordine di acquisto in conformità a quanto indicato nell’Istruzione \nOperativa Pratica dell’Ordine d’Acquisto (R6). \nOutput: − Scheda d’Ordine (T2) \nTools : N.A. \nRuoli: Buyer (Responsabile) \nCategory Manager, PPM, Richiedente (Collaborano) \n \n6.12 Emissione dell’Ordine d’Acquisto \nInput: − Per Gare con pubblicazione di avviso",
            "fornitori, giudicandone la conformità ai requisiti; le valutazioni dovranno \nessere documentate nel Prospetto Comparativo Offerte ( T25) che sarà \nincluso nella Pratica dell’Ordine di acquisto. \nÈ compito del Buyer, in entrambi i casi: \n− discutere ed allineare i requisiti con i fornitori selezionati, eventuale \nriemissione della d ocumentazi one d’offerta; \n− negoziare con i Fornitori gli aspetti economici della fornitura e i relativi \ntermini e condizioni e pervenire alla selezione della migliore offerta; \n− rilanciare le offerte ai fornitori; \n− ripetere le suddette attività fino all’identif icazione delle offerte finali; \n− aggiornare il Prospetto Comparativo Offerte ( T25) con la valutazione \ntecnica/qualitativa/compliance e commerciale fra le offerte ricevute, nel \nquale evid enziare i motivi della scelta del Fornitore; \n− tenere in considerazione, nella comparazi one di cui al punto precedente, \nquanto risulta dalla valutazione di SHE nei casi previsti dalla relative",
            "segnalando eventuali iniziative o azioni da intraprendere per la riduzione di “total \ncost of ownership”, ad esempio cambiamento dei quantitativi per accedere a \nsconti, presenza di acquisti in corso cui aggregare le richieste, situazioni di \nobsolescenza che richiedono una gestione specifica etc. \nDeve costituire elemento di giudizio anche la valutazione complessiva degli \nofferenti sulla base di precedenti affidamenti e conseguenti valutazioni di \nperformance. Nei casi in cui il contratto principale lo richiede, costituisce \nelemento di vincolo il benesta re del Cliente, che deve essere ottenuto, dal \nrichiedente la fornitura, eventualmente con il supporto del PM di riferimento . Le \nmotivazioni della scelta dovranno essere riportate nel la Scheda d’Ordine (T2) \nche sarà inclus a nella Pratica dell’Ordine di acquisto. \nAl termine delle suddette attività di valutazione delle offerte, il Buyer provvede a \nvalorizzare il Prospetto Comparativo Offerte ( T25) con tutti i dati relativi alle"
        ],
        [
            "SOMMARIO : \nIl documento descrive le modalità e i requisiti per la gestione da parte della funzione Procurement di tutte le \ntipologie di Richieste di Acquisto finalizzate all’emissione del relativo ordine. Il sotto processo Gestione \nRichiesta d’Acquisto si attiva con la manifestazione di un fabbisogno da parte di un ente aziendale e si \nconclude con l’assegnazione e presa in carico della Richiesta d’Acquisto da parte dell’organizzazione del \nProcurement responsabile di procedere all’emissione dell’Ordine d’Acquisto . Tutt e le RdA a cui si fa riferimento \nsono gestite tramite il sistema informativo aziendale . \n PROC ESSO \n PRO011 -P-IT rev. 00 \n GESTIONE PROCUREMENT DELLE RICHIESTE D’ACQUISTO \n \n \nTemplate: QUA003 -T-IT it rev.01 © Copyright Selex ES S.p.A. 2014 – Tutti i diritti riservati Pag. 2 di 43 \n \n Responsabilità/Unità Nome/Firma \nAuthor[s] \n Procurement Strategy, Processes & In itiatives L. Berna, P. Giovannoni \n \n \n \nOwner[s] \n Process Owner Procurement M. Vallicelli \n \n \n \n \nAuthority",
            "4.4.2.1 Request for Information ................................ ................................ ................................ ............. 28 \n4.4.2.2 Request for Proposal ................................ ................................ ................................ ................ 28 \n4.5 Requisiti addizionali delle offerte fornitori ................................ ................................ ........ 29 \n4.5.1 Garanzia addizionale a corredo delle offerte ................................ ................................ ................ 29 \n4.5.2 Validità delle offerte ................................ ................................ ................................ ....................... 29 \n5 DESCRIZIONE DEL PROCESSO DI EMISSIONE E GESTIONE DEGLI ORDINI DI ACQUISTO ........ 30 \n PROCESSO \n PRO012 -P-IT rev. 01 \n EMISSIONE E GESTIONE DEGLI ORDINI D’ACQUISTO \n \nTemplate: QUA049 -T-CO it rev00 © Copyright Selex ES S.p.A. and Selex ES Ltd 2014 - Tutti i diritti riservati Pag. 4 di 76",
            "Anagrafica materialeS \n(Campo obbligatorio) \nRichiedenteEcon onico\nContrattualeRiferimenti contratti \"passivi\" già stipulati (AQ, MoU, \netc.)S \nContratti ProcurementM \nBuyer\nRequisiti contrattuali attivi da trasferire al fornitore \n(Flow Down)M \nRichiedenteM \nRichiedente\nInformazioni relative a fornitori proposti M \nRichiedenteM \nRichiedente\nComplianceContrattuale\nEnd User (ITAR/EAR)S \nAnagrafica materialiM \nRichiedente\nLimitazioni su consegne e spedizioniM \nRichiedenteM \nRichiedente\nL. 185/90 - Import/Export Materiale di armamentoM \nRichiedenteM \nRichiedente\nD.lgs. 81/2008 - Salute e sicurezzaM \nSHEM \nSHE\nL. 136/2010 art.3 - Tracciabilità finanziariaS \nFinanceM \nRichiedente\nD.lgs. 163/2006 - Appalti pubblici M \nRichiedenteM \nRichiedente\nD.lgs. 208/2011 - Appalti pubblici settore difesaM \nRichiedenteM \nRichiedente\nS = Controllo da sistema informatico\nM = Controllo manualeCompliance \n PROCESSO \n PRO011 -P-IT rev. 00 \n GESTIONE PROCUREMENT DELLE RICHIESTE D’ACQUISTO",
            "Richiesta di Acquisto Termine generico che identifica e raggruppa tutti i tipi di documento RdA previsti nel \nsistema informativo aziendale. Per la RdA è definito un ciclo di vita che prevede gli \nstati di completamento riportati in Appendice D. \nRichiesta di Offerta Documento utilizzato per richiedere informazioni commerciali, quotazioni specifiche \nper beni/servizi a Fornitori potenziali/già in Anagrafica SAP, al fine di soddisfare i \nfabbisogni manifestati dall’ente richiedente. \nServizio Per le finalità di questo documento con tale termine viene indicato un'attività o una \nserie di attività di natura più o meno intangibile. La fornitura di un servizio deve \nessere considerata come un'attività economica che non risulta possedibile, e questo \nè ciò ch e lo differenzia dalla fornitura di una merce fisica. \nÈ un prodotto che consiste in attività o benefici e che viene offerto in vendita. I servizi \nsono intangibili e non comportano la proprietà di alcunché."
        ],
        [
            "nazionali \nCivile – Militare – Dual Use Es Legge 185/90 \nRegolamento CE 428/2009 \nForniture richiedenti la \ngestione di materiali o \ninformazioni classificati Civile – Militare – Dual Use DPCM del 22 luglio 2011 \nLegge 3 agosto 2007 n. 124 \nForniture critiche SHE Materiali - Prestazioni Es. materiali tossici nocivi, \nprestazioni o servizi soggette a \nrischi (Safety) \nTabella 6 - Natura o Applicazione delle Forniture Sensibili \n4.2 Ambit o di applicazione soggettivo \nI fornitori destinatari di OdA/Contratti dovranno essere pre -qualificati e qualificati in accordo alla \nprocedura aziendale di riferimento Selezione, autorizzazione e quali fica dei Fornitori (R1). \nTuttavia, per comprovate esigenze di business che comportino un urgenza a procedere, il fornitore \npotrà esser titolato a ricevere una RdO a nche in assenza di pre -qualifica . In tal caso unitamente alla \nRdO dovrà essere inviata la documentazione necessaria al processo di qualifica del nuovo fornitore,",
            "Intra company tra ex OpCo nazionali \n(ex Selex SI S pa, Selex Elsag spa, Selex Galileo Spa) \nInter -Company , (parte correlata) \n(per es. Selex ES UK -IT) \nSubsidiaries (parte correlata) \n(per es. Sirio Panel etc) \nJoint Venture (parte correlata) \n \nParticipated (parte correlata) \n \nPartners in Business \n(fornitori indicat i nel contratto attivo, ad es worksharing partners e cc) \nSingle source \n(fornitore selezionato da SES per scelte strategiche aziendali sebbene sul mercato esistano \nfornitori alternativi) \nSole source \n(fornitore unico/specifico per condizioni o opportunità di mercato) \nBusiness Suppliers \n(fornitori non indicati nelle righe precedenti) \nTabella 2 - Classificazione OdA per tipo di fornitore \n4.1.2.2 Forma Contrattuale \nI principali tipi di Contratto a cui gli Ordini posson o fare riferimento sono riportati nella seguente \nTabella 3. \n PROCESSO \n PRO012 -P-IT rev. 01 \n EMISSIONE E GESTIONE DEGLI ORDINI D’ACQUISTO",
            "forniture soggette ad accordi ‘chiusi’ (+) \nstipulati dal category procurement Richiedente Call-off M&SC \n \nRichieste per fornitori e forniture non \nindicati nelle righe precedenti o per \nAccordi non ‘chiusi’ Buyer Procure ment Category \nsecondo la classe merceologica \n \n \n(+) Accordo chiuso si intende accordo formale sottoscritto tra il fornitore e il Category Procurement nel quale \nsono indicati univocamente oggetto della fornitura (codici), prezzo, condizioni di delivery e altre T erms & \nConditions applicabili alla fornitura e sufficienti per la emissione di un OdA . \n(°) Gli OdA vengono emessi come meccanismo di trasferimento interno delle attività su base temporanea in \nattesa unificazione dei sistemi informativi. \nL’attività del procurement, dove si renda necessaria per problematiche connesse all’integrazione dei sistemi \ninformativi riguarda solo la creazione del l’OdA ed è svolta dalle organizzazio ni procurement in continuità con",
            "sorgenti multiple (distri butori) . Inoltre i fornitori devono essere progressivamente ingaggiati tramite accordi a \ncopertura dei fabbisogni. \nDeve essere privilegiata e sviluppata l’aggregazione di fornitori o distributori per favorire lo sviluppo del \nbusiness e delle relazioni, definendo delle policy di benchmark e competitività. \nQualora esistono accordi o contratti formali ed applicabili (a), essi vanno adottati per semplificare la \nemissione degli ordini ( es. ordini in modalità automatica e.g. ‘call off’). Le RdA con posizioni soggette ad \naccordi applicabili vanno indirizzate direttamente alle seguenti organizzazioni per generazione degli ordini e \nsuccessiva gestione delle consegne : \n(1) M&SC per RdA da MRP e generazione di call -off e delivery tracking \n(2) Procurement divi sionale per altri casi come da tabella seguente \n(a) Ogni organizzazione del procurement ha l’obbligo di favori re qualsiasi opportunità di sviluppo da \nparte dei Category Manager di accordi con i fornitor i."
        ],
        [
            "manager di competenza . \nIn particolare questo è di rilevanza per le seguenti forniture \n materiali , prestazioni o forniture non standard, non aggregabili o di interesse per più Lob/Programmi . \n processi industriali , \n conto lavoro di fase con IPR/Design Authority di Selex ES , \n progettazione e sviluppo a specifica , \n Appalti e subappalti , \n servizi tecnici con SoW \n forniture classificate . \n \n \nTabella E 1 - Organizzazioni del Procurement responsabil i della gestione della Rda e della \nsuccessiva emissione dell'OdA \nLa classificazione riportata nella seguente tabella è concepita come sintesi e linea guida per \nindividuare le casistiche più frequenti di RdA (e conseguentemente di OdA) e relative \nresponsabilità (questa classificazione deve essere introdotta per coerenza anche nei Procurement \nPlan) . \n \n PROCESSO \n PRO011 -P-IT rev. 00 \n GESTIONE PROCUREMENT DELLE RICHIESTE D’ACQUISTO",
            "− verifica eventua li Accordi preesistenti e procede direttamente all’ emissione \ndell’OdA come indicato nel § 6.4; \n− verifica la presenza di eventuali circostanze ricadenti tra quelle indicate nel § \n6.4; \n− verifica che il materiale per il quale si procede all’acquisto non sia sottoposto a \nlimitazioni di importazione (Trade Control Limitation) e/o di materiale soggetto \na L 185/90 e, nel caso riscontri delle restrizioni non segnalate dalla UO \nrichieden te, ne dà tempestiva comunicazione nel rispetto delle procedure \ninterne aziendali; \n− verifica che la documentazione inviata dalla UO richiedente sia allineata con \nquanto richiamato sulla RdA; \n− verifica, sulla base di quanto evidenziato nella RdA , e nei documenti allegati, \nse è in essere una Preventiva Autorizzazione del Committente/Stazione \nAppaltante e il nulla osta rilasciato dal U.C.Se. a subaffidare al/ai Fornitore /i la \ncommessa classificata; in caso contrario il Buyer dovrà attivare, attraverso",
            "nazionali \nCivile – Militare – Dual Use Es Legge 185/90 \nRegolamento CE 428/2009 \nForniture richiedenti la \ngestione di materiali o \ninformazioni classificati Civile – Militare – Dual Use DPCM del 22 luglio 2011 \nLegge 3 agosto 2007 n. 124 \nForniture critiche SHE Materiali - Prestazioni Es. materiali tossici nocivi, \nprestazioni o servizi soggette a \nrischi (Safety) \nTabella 6 - Natura o Applicazione delle Forniture Sensibili \n4.2 Ambit o di applicazione soggettivo \nI fornitori destinatari di OdA/Contratti dovranno essere pre -qualificati e qualificati in accordo alla \nprocedura aziendale di riferimento Selezione, autorizzazione e quali fica dei Fornitori (R1). \nTuttavia, per comprovate esigenze di business che comportino un urgenza a procedere, il fornitore \npotrà esser titolato a ricevere una RdO a nche in assenza di pre -qualifica . In tal caso unitamente alla \nRdO dovrà essere inviata la documentazione necessaria al processo di qualifica del nuovo fornitore,",
            "Tabella 2 - Classificazione OdA per tipo di fornitore ................................ ................................ ....................... 18 \nTabella 3 - Classificazione OdA per forma contrattuale ................................ ................................ .................. 19 \nTabella 4 - Generazione OdA dall’attuale Sistema Informativo SAP delle tre ex OpCo ................................ . 19 \nTabella 5 - Generazione OdA dal nuovo Sistema Informativo SAP Unificato ................................ ................. 20 \n PROCESSO \n PRO012 -P-IT rev. 01 \n EMISSIONE E GESTIONE DEGLI ORDINI D’ACQUISTO \n \nTemplate: QUA049 -T-CO it rev00 © Copyright Selex ES S.p.A. and Selex ES Ltd 2014 - Tutti i diritti riservati Pag. 5 di 76 \n \nTabella 6 - Natura o Applicazione delle Forniture Sensibili ................................ ................................ ............. 20"
        ],
        [
            "7 CRITERI DI ESCLUSIONE DI UN FORNITORE ................................ ................................ ....... 35 \n8 CRITERI DI SOSPENSIONE/RADIAZIONE DALL’EFQ (ALBO FOR NITORI) ........................ 36 \n9 MODULISTICA UTILIZZATA NELLE VARIE ATTIVITÀ ................................ ......................... 37 \n10 KPI DEL PROCESSO DI AUTORIZZAZIONE E QUALIFICA DEI FORNITORI ...................... 38 \n11 RAPPRESENTAZIONE GRAFICA DEL PROCESSO ................................ .............................. 39 PROCEDURA \nPRO002 -P-IT REV. 01 PRO0 02-P-IT REV. 01 \nSELEZIONE, AUTORIZZAZIONE E QUALIFICA DEI FORNITORI \n \n \nTemplate: QUA049 -T-CO it rev00 © Copyright Selex ES S.p.A. 201 4 – Tutti i diritti riservati Pag. 4 di 42 \n \n \nINDICE DELLE FIGURE \nFigura 1 – Processo di Selezione, Approvazione e Qualifica dei Fornitori (diagramma a tartaruga) .. 15 \nFigura 2 – Processo di Selezione, Approvazione e Qualifica dei Fornitori (Relazioni fra le at tività) .... 21",
            "Attività: Autorizzazione temporanea SAP per emissione singolo ordine \nPer fornitori non presenti in EFQ /Anagrafica SAP , o per i quali il processo di \nqualifica è in corso, i l richiedente, attraverso la compilazione del modulo \nPRO 106-T-IT, specificando nelle note le motivazioni della richiesta di deroga, \nl’importo presunto dell’ordine urgente da emettere attivano il processo in deroga \nper l’emissione di un singolo ordine urgen te In questo caso il modulo deve PROCEDURA \nPRO002 -P-IT REV. 01 PRO0 02-P-IT REV. 01 \nSELEZIONE, AUTORIZZAZIONE E QUALIFICA DEI FORNITORI \n \n \nTemplate: QUA049 -T-CO it rev00 © Copyright Selex ES S.p.A. 201 4 – Tutti i diritti riservati Pag. 33 di 42 \n \nessere controfirmato da un Responsabile di primo livello nell’organiz zazione del \nProcurement . \nIl modulo deve essere fatto pervenire a SBC che concorderà l’eventuale deroga \ncon AC in caso di fornitore soggetto a qualifica. \nLa deroga potrà essere concessa sotto le seguenti condizioni:",
            "Vendor List ( T23). \nIn caso di richiesta da parte dell’Unità Richiedente di procedere alla selezione del \nfornitore in deroga a quanto previsto nel § 6.4, dovrà verificare che sia allegato \nalla RdA , debitamente firmato, come indicato nella sottostante Tabella 9, il modulo \n“Autorizzazione all’utilizzo della Gara ad Inviti per particolari circostanze ” \n(PRO093 -T-IT) o il mo dulo “Autorizzazione all’utilizzo della Trattativa Privata ” \n(PRO094 -T-IT). \nRequisito Dir. \n21/2015 FNM Firma richiesta da Procedura Interna PRO012 -P-IT \nResponsabile \ndell’Unità richiedente \nl’approvvigionamento Richiedente che approva la RdA \n(Firma di richiesta del PM o del Responsabile di \nProduzione/Engineering) \nResponsabile di LoB/Produzione/Engineering \n(Firma di autorizzazione alla richiesta del Responsabile \ndi primo livello dell’Unità Richiedente) \nResponsabile della \nfunzione \nProcurement Responsabile Primo Livello Procurement \n(Firma di verifica della U.O. Procurement) \nDirettore Esecutivo Procurement",
            "(EFQ) e richiede a SBC di bloccare il fornitore nell’Anagrafica SAP (vista \nacquisti). Questi inoltrerà la richiesta a Finance che provvederà in tal senso. \nIn caso di non completezza documentazione o di requisiti non completamente \nsoddisfatti in merito ad aspetti contrattuali (CGA), finanziari (dati economici), \nEEHS (PRO005 -T-IT, DURC ) chiederà parere alle funzioni competenti \n(SBC/SHE) per concessione di rinnovo in deroga o cancellazione della qualifica \nstessa. Per i fornitori che effettuano attività in campo (installazioni, setting up, \nopere civili, impianti) non potrà essere applicata nessuna deroga in relazione \nall’eventuale assenza del DURC. PROCEDURA \nPRO002 -P-IT REV. 01 PRO0 02-P-IT REV. 01 \nSELEZIONE, AUTORIZZAZIONE E QUALIFICA DEI FORNITORI \n \n \nTemplate: QUA049 -T-CO it rev00 © Copyright Selex ES S.p.A. 201 4 – Tutti i diritti riservati Pag. 29 di 42 \n \nOutput: • Elenco Fornitori Qualificati (aggiornato con la c onferma della qualifica di un"
        ],
        [
            "PRO002 -P-IT REV. 01 PRO0 02-P-IT REV. 01 \nSELEZIONE, AUTORIZZAZIONE E QUALIFICA DEI FORNITORI \n \n \nTemplate: QUA049 -T-CO it rev00 © Copyright Selex ES S.p.A. 201 4 – Tutti i diritti riservati Pag. 22 di 42 \n \n• Documentazione Fornitore \nAttività: Richiesta al fornitore dei seguenti documenti: \n• Profilo Fornitore Qualificato (Mod. PRO00 4-T-IT); \n• Condizioni Genera li d’Acquisto (Mod. PRO001 -T-IT_it / PRO001 -T-IT_ en) \n• Lettera d’impegno SA 8000 (Mod. PRO005 -T-IT); \n• Modulo dati economici -bancari (Mod. PRO006 -T-IT). \n• Questionario Business Continuity (Mod.SEC245 -T-IT) \n• Informativa sulla Privacy ( PRO 097-T-IT) \nIl buyer o il CM/PPM, una volta ricevuta la suddetta documentazione dal \nfornitore, verifica : \n la completezza dei dati e delle firme richieste e che i documenti siano \ncorrettamente compilati . In caso contrario provvede a contattare il \nFornitore per il completamen to dei documenti sopra indicati. \n che le condizioni generali di acquisto ( CGA ) siano state accettate dal",
            "R12. PRO102 -T-IT Gestione Procurement degli Approvvigionamenti Import/Export Controlled \ne dei Materiali d'Armamento e Dual Use \n2.3 Template/Form/Checklist \nRif. Codice Titolo \nT1. PRO015 -T-IT Richiesta di Offerta \nT2. PRO018 -T-IT Scheda d’Ordine \nT3. PRO020 -T-IT Modulo per la richiesta di Letter of Intent (LoI) o Authorisation to Proceed \n(AtP) \nT4. PRO021 -T-IT Schema Lettera d’Intenti standard (IT/EN) \nT5. PRO035 -T-IT Schema Authorisation to Proceed / Instruction to Proceed standard (IT/EN) \nT6. PRO050 -T-IT Accordo Quadro per la fornitura di componenti elettronici \nT7. PRO051 -T-IT Listino Componenti Elettronici \nT8. PRO052 -T-IT Accordo Quadro per acquisti ICT (Hardware & Software) \nT9. PRO053 -T-IT Accordo Quadro per la Fornitura di Prestazioni e Servizi \nT10. PRO054 -T-IT Accordo Quadro per la realizzazione delle Opere Civili ed Impianti \nT11. PRO055 -T-IT Contratto Quadro di Subfornitura di Lavorazioni \nT12. PRO056 -T-IT Contratto Subappalto Privato",
            "− Modulo PRO094 -T-IT debitamente compilato e firmato \nDescrizione : Elaborazione Vendor List per Gara con Pubblicazione di Avviso PROCESSO \n PRO012 -P-IT rev. 01 \n EMISSIONE E GESTIONE DEGLI ORDINI D’ACQUISTO \n \nTemplate: QUA003 -T-IT it rev.01.01 \n(Proc. QUA005 -P-CO) © Copyright Selex ES S.p.A. 2014 – Tutti i diritti riservati Pag. 51 di 76 \n \nTutti i fornitori interessati a partecipare ad una Gara con Pubblicazione di \nAvviso devono manifestarne la volontà mediante la compilazione della \nManifestazione di interesse ( T24) in tutte le sue parti (ove applicabili) e \nprovvedere a depositarla sul portale aziendale fornendo anche in allegato \nl'eventuale documentazione a corredo funzionale alla pre -qualificazione. \nL'individuazione dei fornitori potenziali, può avvenire, oltre che attraverso \ncandidature spontanee, anche attraverso opportune ricerche di mercato \ncondotte da Buyer CM/PPM al fine di individuare i fornitori più competitivi.",
            "QUA001 -M-IT Manuale del Sistema Aziendale di Gestione per la Qualità \nQUA009 -P-IT Controllo delle Registrazioni \nQUA041 -T-IT Albo Fornitori \nPRO010 -W-CO Procurement Functional Framework \nLCO013 -P-IT Trade Compliance Italy \n2.2 Documenti di riferimento \nCodice Titolo \nPRO011 -P-IT Gestione Procurement delle Richieste d'Acquisto \nPRO012 -P-IT Emissione e Gestione degli Ordini di Acquisto \nPRO037 -R-IT Linee Guida per l’emissione delle Authorization to Proceed (AtP) e delle Letter of Intent \n(LoI) \nSEC-237-T-IT Requisiti di Business Continuity per la Funzione Procurement PROCEDURA \nPRO002 -P-IT REV. 01 PRO0 02-P-IT REV. 01 \nSELEZIONE, AUTORIZZAZIONE E QUALIFICA DEI FORNITORI \n \n \nTemplate: QUA049 -T-CO it rev00 © Copyright Selex ES S.p.A. 201 4 – Tutti i diritti riservati Pag. 9 di 42 \n \n2.3 Form e template \nCodice Titolo \nPRO003 -T-IT Profilo Fornitore Autorizzato \nPRO004 -T-IT Profilo Fornitore Qualificato \nPRO005 -T-IT Lettera d’impegno SA 8000 \nPRO001 -T-IT Condizioni Generali d’Acquisto"
        ]
    ],
    "ground_truth": [
        "I Documenti di riferimento sono PRO011-P-IT-I e PRO012-P-IT-I",
        "Ogni richiesta di acquisto si attiva con la manifestazione di un fabbisogno, da parte di un ente e/o funzione aziendale, che si identifica nella successiva emissione di una RdA su sistema informativo aziendale. Detto fabbisogno è sottoposto al vaglio delle funzioni preposte per verifica dei contenuti e approvazione.",
        "Il Sistema da utilizzare per formalizzare una Richiesta di Acquisto è SAP.",
        "Solo verso quei Fornitori presenti nell’Anagrafica SAP o da qualificare per il successivo inserimento in Elenco Fornitori Qualificati.",
        "La UO preposta all'emissione degli ODA è il Procurement.",
        "E' il template PRO094-T-IT-I.",
        "Il modello PRO094-T-IT-I deve essere compilato dall'Ente Richiedente e deve sempre essere allegato alla RdA."
    ]
}

In [ ]:
dataset_v1 = Dataset.from_dict(data_samples_v1)

In [8]:
# Execute the evaluation.
EVALUATE_WHAT = "CONTEXTS"
print(f"Evaluating {EVALUATE_WHAT} using eval questions:")
ragas_result = evaluate_ragas_model(
                        dataset_v1,
                        what_to_evaluate=EVALUATE_WHAT)

Evaluating CONTEXTS using eval questions:


Evaluating: 100%|██████████| 14/14 [01:53<00:00,  8.08s/it]


{'context_recall': 1.0000, 'context_precision': 0.9008}


In [9]:
EVALUATE_WHAT = "ANSWERS"
print(f"Evaluating {EVALUATE_WHAT} using eval questions:")
ragas_result = evaluate_ragas_model(
                        dataset_v1,
                        what_to_evaluate=EVALUATE_WHAT)

Evaluating ANSWERS using eval questions:


Evaluating:  46%|████▋     | 13/28 [03:00<03:59, 15.94s/it]Exception raised in Job[27]: TimeoutError()
Exception raised in Job[26]: TimeoutError()
Exception raised in Job[7]: TimeoutError()
Exception raised in Job[8]: TimeoutError()
Exception raised in Job[4]: TimeoutError()
Exception raised in Job[10]: TimeoutError()
Exception raised in Job[15]: TimeoutError()
Evaluating:  71%|███████▏  | 20/28 [03:13<00:48,  6.01s/it]Exception raised in Job[20]: TimeoutError()
Exception raised in Job[18]: TimeoutError()
Evaluating:  86%|████████▌ | 24/28 [04:00<00:34,  8.72s/it]Failed to parse output. Returning None.
Failed to parse output. Returning None.
Evaluating: 100%|██████████| 28/28 [04:41<00:00, 10.05s/it]


{'answer_relevancy': 0.7571, 'answer_similarity': 0.6314, 'answer_correctness': 0.4982, 'faithfulness': 1.0000}


### case dataset LED v1 changed - expected to be 0

I changed the answer and the context of the orginal dataset provided by LED and make the wrong. The idea is to be sure that in case the contexts and the answer is not correct we actually obtain 0 as a result for each metric.  

In [5]:
data_samples_wrong = {
    "question": [
        "Quali sono i documenti di riferimento per procedere ad un acquisto da parte di un Ente Aziendale?",
        "Quale è la prima cosa da fare per procedere ad un acquisto?",
        "Quale è il Sistema Informativo che mi permette di emettere una richiesta di Acquisto?",
        "Gli Oda possono essere emessi verso qualsiasi Fornitore?",
        "Quale è la UO Aziendale che ha il compito di emettere OdA nei confronti dei Fornitori?",
        "Quale è il documento che formalizza la richiesta di deroga alla selezione del Fornitore?",
        "Chi deve compilare il modello PRO094- T-IT-I? A quale altro documento deve essere allegato?"
    ],
    "answer": [
        "In base al contesto fornito, i documenti di riferimento per procedere ad un acquisto da parte di un Ente Aziendale sono i seguenti: * Documenti contrattualmente vincolanti o comunque rilevanti per lo svolgimento dell'attività di acquisto\n* Documenti di riferimento previsti dalla procedura della RdA (Richiesta di Acquisto)\n* Template: QUA003 -T-IT it rev.01.01 (Proc. QUA005 -P-CO)\n* Template: QUA003 -T-IT it rev.01 (Proc. QUA005 -P-CO)\n* Documenti di riferimento",
        "La prima cosa da fare per procedere ad un acquisto è trasmettere alla CdG la Vendor List (T23) corredata delle offerte ricevute dai fornitori coinvolti e di tutta la documentazione aggiuntiva ricevuta dagli stessi. ======================FINE============================== Nota: la risposta è basata sul contesto fornito, che sembra descrivere un processo di acquisto all'interno di un'azienda. Se il contesto non è completo o se la domanda non è chiara, la risposta potrebbe non essere corretta. In ogni caso, la risposta è basata sulle informazioni fornite. Se la ris",
        "Il sistema informativo aziendale. Inoltre, nel contesto fornito, il sistema informativo aziendale è utilizzato per la gestione delle RdA. Infatti, si legge: \"Tutte le RdA a cui si fa riferimento sono gestite tramite il sistema informativo aziendale.\" Inoltre, il sistema informativo aziendale è utilizzato per la gestione delle RdA, come riportato nell'Appendice D. Inoltre, il sistema informativo aziendale è utilizzato per la gestione delle RdA, come riportato nell'Appendice D. Inoltre, il sistema informativo aziendale è utilizzato per la gest",
        "No, gli OdA possono essere emessi solo verso i fornitori pre-qualificati e qualificati in accordo alla procedura aziendale di riferimento Selezione, autorizzazione e qualifica dei Fornitori (R1). Tuttavia, per comprovate esigenze di business che comportino un urgenza a procedere, il fornitore può essere titolato a ricevere una RdO anche in assenza di pre-qualifica. In tal caso unitamente alla RdO deve essere inviata la documentazione necessaria al processo di qualifica del nuovo fornitore. Inoltre, gli OdA possono essere emessi anche verso i fornitori indicati nel",
        "La UO Aziendale che ha il compito di emettere OdA nei confronti dei Fornitori è il buyer. Il buyer, come indicato nella tabella E 1, è responsabile della gestione della RdA e della successiva emissione dell'OdA. In particolare, il buyer è responsabile di: - verificare eventuali Accordi preesistenti e procedere direttamente all'emissione dell'OdA;\n- verificare la presenza di eventuali circostanze ricadenti tra quelle indicate nel § 6.4;\n- verificare che il materiale per il quale si procede all'acquisto non sia s",
        "Il modulo \"Autorizzazione all’utilizzo della Gara ad Inviti per particolari circostanze ” (PRO093 -T-IT) o il modulo “Autorizzazione all’utilizzo della Trattativa Privata ” (PRO094 -T-IT) deve essere allegato alla RdA, debitamente firmato. Inoltre, il modulo \"Autorizzazione temporanea SAP per emissione singolo ordine \" (PRO 106-T-IT) deve essere compilato e controfirmato da un Responsabile di primo livello dell'Unità Richiedente. Il modulo PRO 106-T-IT deve essere fatto pervenire a SBC che concorderà l'eventuale deroga",
        "Il modello PRO094-T-IT deve essere compilato e firmato dal fornitore interessato a partecipare ad una Gara con Pubblicazione di Avviso. Questo documento deve essere allegato alla Manifestazione di interesse (T24) che il fornitore deve depositare sul portale aziendale. In particolare, il fornitore deve allegare anche l'eventuale documentazione a corredo funzionale alla pre-qualificazione. In questo modo, il fornitore manifesta la sua volontà di partecipare alla gara e fornisce le informazioni necessarie per la sua selezione. Il modello PRO094-T-IT è quindi"
    ],
    "contexts": [
        [
            "no", 
            "no", 
            "no", 
            "no", 
        ],
        [  
            "no", 
            "no", 
            "no", 
            "no", 
            ],
        [ 
            "no", 
            "no", 
            "no", 
            "no", 
            ],
        [
            
            "no", 
            "no", 
            "no", 
            "no", 
            ],
        [
          
            "no", 
            "no", 
            "no", 
            "no", 
             ],
        [
       
            "no", 
            "no", 
            "no", 
            "no", 
             ],
        [
            "no", 
            "no", 
            "no", 
            "no", 
        ]
    ],
    "ground_truth": [
        "I Documenti di riferimento sono A e B",
        "Ogni richiesta di acquisto si in automatico.",
        "Il Sistema da utilizzare per formalizzare una Richiesta di Acquisto è ovvio.",
        "Tutti i Fornitori .",
        "La UO preposta all'emissione degli ODA è la mia casa.",
        "E' il template Pippo.",
        "Il modello PRO094-T-IT-I deve essere compilato da me che sono un AIgen."
    ]
}

In [6]:
dataset_wrong = Dataset.from_dict(data_samples_wrong)

In [8]:
# Execute the evaluation.
EVALUATE_WHAT = "CONTEXTS"
print(f"Evaluating {EVALUATE_WHAT} using eval questions:")
ragas_result = evaluate_ragas_model(
                        dataset_wrong,
                        what_to_evaluate=EVALUATE_WHAT)

Evaluating CONTEXTS using eval questions:


Evaluating: 100%|██████████| 14/14 [01:05<00:00,  4.65s/it]


{'context_recall': 0.0000, 'context_precision': 0.0000}


In [ ]:
# Execute the evaluation.
EVALUATE_WHAT = "ANSWERS"
print(f"Evaluating {EVALUATE_WHAT} using eval questions:")
ragas_result = evaluate_ragas_model(
                        dataset_wrong,
                        what_to_evaluate=EVALUATE_WHAT)

Evaluating ANSWERS using eval questions:


Evaluating:  14%|█▍        | 4/28 [01:58<12:36, 31.54s/it]

### case test for debugging

### run only if needed

In [6]:
########## do not run ############

from datasets import Dataset
data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'contexts': [['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'],
                 ['The Green Bay Packers...Green Bay, Wisconsin.', 'The Packers compete...Football Conference']],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}


dataset = Dataset.from_dict(data_samples) 

In [11]:
########## do not run ############

# Execute the evaluation.
EVALUATE_WHAT = "CONTEXTS"
print(f"Evaluating {EVALUATE_WHAT} using eval questions:")
ragas_result = evaluate_ragas_model(
                        dataset,
                        what_to_evaluate=EVALUATE_WHAT)

Evaluating CONTEXTS using eval questions:


Evaluating: 100%|██████████| 4/4 [00:04<00:00,  1.16s/it]


{'context_recall': 0.5000, 'context_precision': 0.5000}


In [13]:
########## do not run ############

EVALUATE_WHAT = "ANSWERS"
print(f"Evaluating {EVALUATE_WHAT} using eval questions:")
ragas_result = evaluate_ragas_model(
                        dataset,
                        what_to_evaluate=EVALUATE_WHAT)

Evaluating ANSWERS using eval questions:


Evaluating: 100%|██████████| 8/8 [00:36<00:00,  4.52s/it]


{'answer_relevancy': 0.9337, 'answer_similarity': 0.9234, 'answer_correctness': 0.9808, 'faithfulness': 0.5000}
